In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.metrics import accuracy_score
import random
import csv

personality_data = pd.read_csv('./data/PersonalityData.csv', names=["user_id", "openness", "agreeableness", "emotional_stability", "conscientiousness", "extraversion"])
print("pdf", personality_data.shape)
rating_data = pd.read_csv('./data/Rating.csv', names=["user_id", "movie_id", "rating", "timestamp"])
rating_data = rating_data.drop("timestamp", axis=1)
movietag_data = pd.read_csv('./data/MovieTag.csv', names=["movie_id", "tag_id"])
tag_data = pd.read_csv('./data/Tag.csv', names=["tag_id", "tag"])

print("rdf", rating_data.shape)
whole_data = pd.merge(personality_data, rating_data, left_on='user_id', right_on='user_id', how='inner')
whole_data = pd.merge(whole_data, movietag_data, left_on = 'movie_id', right_on='movie_id', how='inner')
whole_data = pd.merge(whole_data, tag_data, left_on = 'tag_id', right_on='tag_id', how='inner')
print(whole_data.head())

pdf (1820, 6)
rdf (912643, 3)
                            user_id  openness  agreeableness  \
0  8e7cebf9a234c064b75016249f2ac65e       5.0            2.0   
1  b7e8a92987a530cc368719a0e60e26a3       4.0            3.0   
2  92561f21446e017dd6b68b94b23ad5b7       5.5            5.5   
3  030001ac2145a938b07e686a35a2d638       5.5            5.5   
4  36f31ef9879b4de96977aeba94878581       5.5            4.5   

   emotional_stability  conscientiousness  extraversion  movie_id  rating  \
0                  3.0                2.5           6.5         1     5.0   
1                  4.5                2.0           2.5         1     3.5   
2                  4.0                4.5           4.0         1     2.0   
3                  3.5                4.5           2.5         1     3.0   
4                  6.0                3.0           4.0         1     4.5   

   tag_id    tag  
0      57  pixar  
1      57  pixar  
2      57  pixar  
3      57  pixar  
4      57  pixar  


In [2]:
tag_map = {}
for index, row in tag_data.iterrows():
    tag_map[row["tag"]] = row["tag_id"]
print(tag_map)

{'funny': 0, 'highly quotable': 1, 'will ferrell': 2, 'boxing story': 3, 'mma': 4, 'tom hardy': 5, 'drugs': 6, 'leonardo dicaprio': 7, 'martin scorsese': 8, 'way too long': 9, 'al pacino': 10, 'gangster': 11, 'mafia': 12, 'holocaust': 13, 'true story': 14, 'twist ending': 15, 'anthony hopkins': 16, 'courtroom drama': 17, 'britpop': 18, 'indie record label': 19, 'music': 20, 'dumpster diving': 21, 'sustainability': 22, 'romantic comedy': 23, 'wedding': 24, 'painter': 25, 'bloody': 26, 'black hole': 27, 'sci-fi': 28, 'time-travel': 29, 'fantasy': 30, 'magic board game': 31, 'robin williams': 32, 'beautiful scenery': 33, 'epic': 34, 'historical': 35, 'inspirational': 36, 'medieval': 37, 'mel gibson': 38, 'oscar (best cinematography)': 39, 'revenge': 40, 'sword fight': 41, 'black comedy': 42, 'christina ricci': 43, 'christopher lloyd': 44, 'dark comedy': 45, 'family': 46, 'gothic': 47, 'andy garcia': 48, 'classic': 49, 'francis ford coppola': 50, 'quirky': 51, 'macaulay culkin': 52, 'seque

In [3]:
tags = list(set(whole_data['tag']))
whole_data['tag'] = whole_data['tag'].str.lower()
tags.remove(tags[0])

def specificMovieData(table, tag):
    return table.loc[table['tag'] == tag]

aggregate_personality = []
# tag_id_to_tag = []

# tag_id = 0

for idx in range(len(tags)):
    tag = tags[idx]

    tag_data = specificMovieData(whole_data,tag)
    tag_data = tag_data.drop('user_id', axis=1)
    tag_data = tag_data.drop('movie_id', axis=1)
   
    tag_data["rating"] = 24 ** tag_data["rating"]
    tag_data["openness"] = tag_data["openness"] * tag_data["rating"]
    tag_data["agreeableness"] = tag_data["agreeableness"] * tag_data["rating"]
    tag_data["emotional_stability"] = tag_data["emotional_stability"] * tag_data["rating"]
    tag_data["conscientiousness"] = tag_data["conscientiousness"] * tag_data["rating"]
    tag_data["extraversion"] = tag_data["extraversion"] * tag_data["rating"]
    
    openness = tag_data["openness"].sum()
    agreeableness = tag_data["agreeableness"].sum()
    emotional_stability = tag_data["emotional_stability"].sum()
    conscientiousness = tag_data["conscientiousness"].sum()
    extraversion = tag_data["extraversion"].sum()
    
    tag_personality = [tag_map[tag.lower()], openness, agreeableness, emotional_stability, conscientiousness, extraversion]
    aggregate_personality.append(tag_personality)
#     tag_id_to_tag.append([tag_id, tag])
    
#     tag_id += 1

#movie_id tag openness agreeableness emotional_stability conscientiousness extraversion
    

In [4]:
with open('./data/TagPersonalities.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(aggregate_personality)
    
# with open('./data/Tags.csv', 'w') as f:
#     w = csv.writer(f)
#     w.writerows(tag_id_to_tag)
    